In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

In [2]:
data = pd.read_csv('/home/mohit/Desktop/ANN_Classification/Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
## Estimated Salary will be the - output feature and remaining all will be the independent
## Preprocessing the data
# 1) drop - CustomerId, Surname and rownumber

data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis = 1)


In [4]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
## Convert - Gender into - 0 and 1 using labelencoder
label_gender = LabelEncoder()
data['Gender'] = label_gender.fit_transform(data['Gender'])

In [6]:
## Convert - Geography into numerical using one hot encoder
one_hot_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded = one_hot_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=one_hot_geo.get_feature_names_out(['Geography']))

In [7]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [8]:
## Combining one-hot encoded columns with original data
data = pd.concat([data.drop('Geography', axis = 1),geo_encoded_df], axis = 1)

In [9]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [10]:
## Split data into features and target

x = data.drop('EstimatedSalary', axis = 1)
y = data['EstimatedSalary']

In [11]:
# Split data into training and testing sets
x_train, x_test, y_train,y_test = train_test_split(x,y,test_size=0.2, random_state=42) 

In [12]:
# Scale the Features
scal = StandardScaler()
x_train = scal.fit_transform(x_train)
x_test = scal.transform(x_test)

In [13]:
## Save the encoder and scaler into pickle file for later use

with open('label_gender.pkl', 'wb') as file:
    pickle.dump(label_gender, file)

with open('one_hot_geo.pkl', 'wb') as file:
    pickle.dump(one_hot_geo, file)

with open('scal.pkl','wb') as file:
    pickle.dump(scal, file)

# ANN Regression Problem Statement

In [14]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input

2024-12-10 01:25:01.195862: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-10 01:25:01.457091: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-10 01:25:01.511477: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733774101.557788   43273 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733774101.577161   43273 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-10 01:25:01.658678: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [15]:
# Building THe Model
model = Sequential([
    Input(shape=(x_train.shape[1],)),  # Define the input layer
    Dense(64, activation='relu'),
    Dense(32, activation ='relu'),
    Dense(1) # output layer - regression , default activation function = Linear activation
])

W0000 00:00:1733774112.304779   43273 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [16]:
# Compile the model
model.compile(optimizer='adam', loss = 'mean_absolute_error', metrics=['mae'])

In [17]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [18]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

# Saving logs
log_dir = "regression/fits/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callbacks = TensorBoard(log_dir= log_dir, histogram_freq=1)

In [19]:
# Set up Early Stopping
early_stopping_callbacks = EarlyStopping(monitor='val_loss', patience=10,restore_best_weights=True)


In [20]:
# Train The Model
history=model.fit(
    x_train,y_train,
    validation_data=(x_test, y_test),
    epochs = 100,
    callbacks=[early_stopping_callbacks, tensorboard_callbacks]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - loss: 100639.6562 - mae: 100639.6562 - val_loss: 98489.0938 - val_mae: 98489.0938
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 98609.4219 - mae: 98609.4219 - val_loss: 96761.0000 - val_mae: 96761.0000
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 97724.6953 - mae: 97724.6953 - val_loss: 92347.9062 - val_mae: 92347.9062
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 92038.6328 - mae: 92038.6328 - val_loss: 85110.4453 - val_mae: 85110.4453
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 83818.3359 - mae: 83818.3359 - val_loss: 75992.4531 - val_mae: 75992.4531
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 74221.3125 - mae: 74221.3125 - val_loss: 66965.2344 - val_mae: 66965.2344
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 65298.2305 - mae: 65298.2305 - val_loss: 59528.5938 - val_mae: 59528.5938
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step 

In [21]:
%load_ext tensorboard

In [22]:
%tensorboard --logdir regression/fits

Launching TensorBoard...

In [23]:
## Evaluate Model on Test Data
test_loss, test_mae = model.evaluate(x_test, y_test)
print(f"Test MAE : {test_mae}")

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 51135.0156 - mae: 51135.0156
Test MAE : 50262.57421875


In [24]:
model.save('regression_model.h5')